# Script GKA
O código abaixo busca integrar todas as planilhas utilizadas no Relatório GKA.  

In [ ]:
import pandas as pd
from settings import Settings
import utils
from rich import print, emoji
import sys
from datetime import datetime

cfg = Settings()

files = {
    "base": cfg.base_file,
    "cadastro": cfg.cadastro_file,
    "gka_por_segmento": cfg.gka_segmento_file,
    "lista_gka": cfg.lista_gka_file,
    "portfolio": cfg.portfolio_file,
    "oem": cfg.oem_file,
    "sellin": cfg.sellin_file

}

dfs = {}

try:
    for key, path in files.items():
        dfs[key] = utils.read_safe_excel(path)
        print(f":white_check_mark: {path} aberto com sucesso! :unicorn_face:")
        
except (FileNotFoundError, PermissionError) as e:
    print(f":cross_mark: Erro ao carregar '{path}': {e}")
    sys.exit(1)

In [ ]:
dfs["oem"] = dfs["oem"].pipe(
    utils.clean_key(["IBM DODO", "Material", "Dsc Segmento Mercado"])
)
dfs["cadastro"] = dfs["cadastro"].pipe(
    utils.clean_key("IBM")
)
dfs["portfolio"] = dfs["portfolio"].pipe(
    utils.clean_key("SKU")
)
dfs["gka_por_segmento"] = dfs["gka_por_segmento"].pipe(
    utils.clean_key("Segmento")
)

working_file = (
    dfs["oem"]
    .merge(dfs["cadastro"][["IBM", "Nom Cliente","KAM"]], left_on="IBM DODO", right_on="IBM", how="left")
    .merge(dfs["portfolio"][["SKU", "Tipo de Embalagem"]], left_on="Material", right_on="SKU", how="left")
    .merge(dfs["gka_por_segmento"][["Segmento", "GKA"]], left_on="Dsc Segmento Mercado", right_on="Segmento", how="left")
    .drop(columns=["IBM", "SKU", "Segmento"])
)

In [ ]:
dfs["sellin"] = (
    dfs["sellin"]
    .pipe(utils.clean_key(["Material", "IBM"]))
)
dfs["lista_gka"] = dfs["lista_gka"].pipe(
    utils.clean_key("IBMs GKA")
)

dfs["sellin"] = dfs["sellin"][dfs["sellin"]["IBM"].isin(dfs["lista_gka"]["IBMs GKA"])]

sellin_final = (
    dfs["sellin"]
    .merge(dfs["cadastro"][["IBM", "Nom Cliente","KAM"]], left_on="IBM", right_on="IBM", how="left")
    .merge(dfs["portfolio"][["SKU", "Tipo de Embalagem"]], left_on="Material", right_on="SKU", how="left")
    .merge(dfs["gka_por_segmento"][["Segmento", "GKA"]], left_on="Dsc Segmento Mercado", right_on="Segmento", how="left")
    .drop(columns=["IBM", "SKU"])
)

In [ ]:
columns_gka = [
    "Sold-To Party", "Sold-To Party Name", "Ship-To Party", "Ship-To Party Name",
    "Pack Size (litres)", "Product Class Hier", "Cal. year / month", "Volume",
    "UF", "GKA", "Material", "Produto", "Descrição Segmento de mercado", "SKU_Class", "SKU Tipo"
]

df_final = pd.DataFrame(columns=columns_gka)

mapeamento_oem = {
    "IBM DODO": "Sold-To Party",
    "Nom Cliente": "Sold-To Party Name",
    "IBM Cliente": "Ship-To Party",
    "Razão Social": "Ship-To Party Name",
    "Material": "Material",
    "Dsc Segmento Mercado": "Descrição Segmento de mercado",
    "Tipo de Embalagem": "SKU Tipo",
    "Volume OEM": "Volume",
    "UF": "UF",
    "Produto": "Produto",
}

df_oem_final = working_file.rename(columns=mapeamento_oem)
df_oem_final = df_oem_final.merge(
    dfs["portfolio"][["SKU", "Pack e UMG", "Linha", "SPC (Saleable Product Code)", "GPO"]], 
    left_on="Material", 
    right_on="SKU", 
    how="left"
)

df_oem_final["Product Class Hier"] = df_oem_final["SPC (Saleable Product Code)"].fillna(0)
df_oem_final["SKU Tipo"] = df_oem_final["Linha"].fillna("NÃO IDENTIFICADO")
df_oem_final["SKU_Class"] = df_oem_final["GPO"].fillna("NÃO IDENTIFICADO")
df_oem_final["Pack Size (litres)"] = df_oem_final["Pack e UMG"].fillna("NÃO IDENTIFICADO")
data_ref = f"{datetime.now().year}M{datetime.now().month:02}"
df_oem_final["Cal. year / month"] = data_ref

df_oem_final = df_oem_final.reindex(columns=columns_gka)

mapping_sellin = {
    "IBM": "Ship-To Party",
    "Nom Cliente": "Ship-To Party Name",
    "Material": "Material",
    "Dsc Segmento Mercado": "Descrição Segmento de mercado",
    "Tipo de Embalagem": "SKU Tipo",
    "Volume OEM": "Volume",
    "Cod Regiao": "UF",
    "SKU Product Description": "Produto",
}
df_sellin_final = dfs["sellin"].rename(columns=mapping_sellin)
df_sellin_final["Sold-To Party"] = "Direta"
df_sellin_final["Sold-To Party Name"] = "Direta"

df_sellin_final = df_sellin_final.merge(
    dfs["portfolio"][["SKU", "Pack e UMG", "Linha", "SPC (Saleable Product Code)", "GPO"]], 
    left_on="Material", 
    right_on="SKU", 
    how="left"
)

df_sellin_final["Product Class Hier"] = df_sellin_final["SPC (Saleable Product Code)"].fillna(0)
df_sellin_final["SKU Tipo"] = df_sellin_final["Linha"].fillna("NÃO IDENTIFICADO")
df_sellin_final["SKU_Class"] = df_sellin_final["GPO"].fillna("NÃO IDENTIFICADO")
df_sellin_final["Pack Size (litres)"] = df_sellin_final["Pack e UMG"].fillna("NÃO IDENTIFICADO")
data_ref = f"{datetime.now().year}M{datetime.now().month:02}"
df_sellin_final["Cal. year / month"] = data_ref

df_sellin_final = df_sellin_final.reindex(columns=columns_gka)

df_gka = pd.concat([df_sellin_final, df_oem_final], ignore_index=True)

In [ ]:
output_path = cfg.output_path / f"{utils.current_ano_safra()} - {cfg.file_name} {datetime.now().month:02}''{datetime.now().year}.xlsx"
cfg.output_path.mkdir(parents=True, exist_ok=True)

with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    df_gka.to_excel(writer, sheet_name='Base GKA Consolidada', index=False)
    
    workbook  = writer.book
    worksheet = writer.sheets['Base GKA Consolidada']

    (max_row, max_col) = df_gka.shape
    
    worksheet.autofilter(0, 0, max_row, max_col - 1)
    worksheet.freeze_panes(1, 0)
    
    for i, col in enumerate(df_gka.columns):
        column_len = max(df_gka[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.set_column(i, i, column_len)

print(f"\n[bold green]:white_check_mark: Relatório Consolidado Gerado com Sucesso![/bold green]")
print(f"[cyan]Local:[/cyan] {output_path} :unicorn_face:")